## <b><font color='darkblue'>Gemini API: OAuth Quickstart</font></b>
([source](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication_with_OAuth.ipynb)) <b><font size='3ptx'>Some parts of the Gemini API like model tuning and semantic retrieval use OAuth for authentication.</font></b>

If you are a beginner, you should start by using [API keys](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb), and come back to this OAuth guide only when you need it for these features.

<b>To help you get started with OAuth, this notebook shows a simplified approach that is appropriate for a testing environment.</b> For a production environment, learn about [authentication and authorization](https://developers.google.com/workspace/guides/auth-overview) before [choosing the access credentials](https://developers.google.com/workspace/guides/create-credentials#choose_the_access_credential_that_is_right_for_you) that are appropriate for your app.

### <b><font color='darkgreen'>Prerequisites</font></b>
To run this quickstart, you need:
* The [**Google Cloud CLI**](https://cloud.google.com/sdk/docs/install-sdk) installed on your local machine.
* [**A Google Cloud project**](https://developers.google.com/workspace/guides/create-project).

### <b><font color='darkgreen'>Set up your Cloud project</font></b>
To complete this quickstart, you first need to setup your Cloud project.

#### <b>1. Enable the API</b>
Before using Google APIs, you need to turn them on in a Google Cloud project.
* In the Google Cloud console, [enable](https://console.cloud.google.com/flows/enableapi?apiid=generativelanguage.googleapis.com) the Google Generative Language API. If you created an API Key in AI Studio, this was done for you.

#### <b>2. Configure the OAuth consent screen</b>
Next configure the project's OAuth consent screen and add yourself as a test user. If you've already completed this step for your Cloud project, skip to the next section.
1. In the Google Cloud console, go to the [OAuth consent screen](https://console.cloud.google.com/apis/credentials/consent), this can be found under `Menu > APIs & Services OAuth consent screen`.
2. Select the user type **External** for your app, then click Create.
3. Complete the app registration form (<font color='brown'>you can leave most fields blank</font>), then click `Save and Continue`.
4. For now, you can skip adding scopes and click `Save and Continue`. In the future, when you create an app for use outside of your Google Workspace organization, you must add and verify the authorization scopes that your app requires.
5. Add test users:
    - Under Test users, click `Add users`.
    - Enter your email address and any other authorized test users, then click `Save and Continue`.
6. Review your app registration summary. To make changes, click `Edit`. If the app registration looks OK, click `Back to Dashboard`.

#### <b>3. Authorize credentials for a desktop application</b>
To authenticate as an end user and access user data in your app, you need to create one or more OAuth 2.0 Client IDs. A client ID is used to identify a single app to Google's OAuth servers. If your app runs on multiple platforms, you must create a separate client ID for each platform.
1. In the Google Cloud console, go to Credentials, this can be found under `Menu > APIs & Services > Credentials`.
2. Click `Create Credentials > OAuth client ID`.
3. Click `Application type > Desktop app`.
4. In the `Name` field, type a name for the credential. This name is only shown in the Google Cloud console.
5. Click `Create`. The OAuth client created screen appears, showing your new Client ID and Client secret.
6. Click `OK`. The newly created credential appears under `OAuth 2.0 Client IDs`.
7. Click the download button to save the JSON file. It will be saved as `client_secret_<identifier>.json`.

### <b><font color='darkgreen'>Set up application default credentials</font></b>
In this quickstart you will use [**application default credentials**](https://cloud.google.com/docs/authentication/application-default-credentials) to authenticate.

#### <b>Add client secret to Colab secrets</b>
If you need to use OAuth with the Gemini API in Google Colab frequently, it is easiest to add the contents of your `client_secret.json` file into Colab's Secrets manager.
1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel.
2. Create a new secret with the name `CLIENT_SECRET`.
3. Open your `client_secret.json` file in a text editor and copy/paste the content into the `Value` input box of `CLIENT_SECRET`.
4. Toggle the button on the left to allow notebook access to the secret.

Now you can programmatically create the file instead of uploading it every time. The client secret is also available in all your Google Colab notebooks after you allow access.
```python
from google.colab import userdata
import pathlib

pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))
```

#### <b>Set the application default credentials</b>
To convert the `client_secret.json` file into usable credentials, pass its location the <font color='blue'>gcloud auth application-default login</font> command's <font color='violet'>--client-id-file</font> argument. The simplified project setup in this tutorial triggers a `Google hasn't verified this app dialog`. This is normal, choose `Continue`.

You will need to do this step once for every new Google Colab notebook or runtime.

<b><font color='orange'>Note</font></b>: Carefully follow the instructions the following command prints (<font color='brown'>don't just click the link</font>). Also make sure your local <font color='blue'>gcloud --version</font> is the [latest](https://cloud.google.com/sdk/docs/release-notes) to match the version pre-installed in Google Colab.

In [2]:
#!gcloud auth application-default login \
#  --no-browser --client-id-file client_secret.json \
#  --scopes https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning,https://www.googleapis.com/auth/generative-language.retriever

he specific scopes you need depends on the API you are using. For example, looking at the API reference for `tunedModels.create`, you will see:
```
Requires one of the following OAuth scopes:
* https://www.googleapis.com/auth/generative-language.tuning
```

This sample asks for all the scopes for tuning and semantic retrieval, but best practice is to use the smallest set of scopes for security and user confidence.

#### <b>Using the Python SDK with OAuth</b>
The Python SDK will automatically find and use application default credentials.

In [4]:
#!pip install -U -q google-generativeai

Let's do a quick test. Note that you did not set an API key using genai.configure()!

In [1]:
import google.generativeai as genai

print('Available base models:', [m.name for m in genai.list_models()])

Available base models: ['models/chat-bison-001', 'models/text-bison-001', 'models/embedding-gecko-001', 'models/gemini-1.0-pro-latest', 'models/gemini-1.0-pro', 'models/gemini-pro', 'models/gemini-1.0-pro-001', 'models/gemini-1.0-pro-vision-latest', 'models/gemini-pro-vision', 'models/gemini-1.5-pro-latest', 'models/gemini-1.5-pro-001', 'models/gemini-1.5-pro', 'models/gemini-1.5-pro-exp-0801', 'models/gemini-1.5-pro-exp-0827', 'models/gemini-1.5-flash-latest', 'models/gemini-1.5-flash-001', 'models/gemini-1.5-flash-001-tuning', 'models/gemini-1.5-flash', 'models/gemini-1.5-flash-exp-0827', 'models/gemini-1.5-flash-8b-exp-0827', 'models/embedding-001', 'models/text-embedding-004', 'models/aqa']


In [3]:
tunable_models = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods]
tunable_models

[Model(name='models/gemini-1.0-pro-001',
       base_model_id='',
       version='001',
       display_name='Gemini 1.0 Pro 001 (Tuning)',
       description=('The best model for scaling across a wide range of tasks. This is a stable '
                    'model that supports tuning.'),
       input_token_limit=30720,
       output_token_limit=2048,
       supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
       temperature=0.9,
       max_temperature=None,
       top_p=1.0,
       top_k=None),
 Model(name='models/gemini-1.5-flash-001-tuning',
       base_model_id='',
       version='001',
       display_name='Gemini 1.5 Flash 001 Tuning',
       description='Fast and versatile multimodal model for scaling across diverse tasks',
       input_token_limit=16384,
       output_token_limit=8192,
       supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
       temperature=1.0,
       max_temperature=2.0,
       top_p=0.95,

In [4]:
base_model = [m for m in tunable_models if 'flash' in m.name][-1]  # models/gemini-1.5-flash-001-tuning
base_model

Model(name='models/gemini-1.5-flash-001-tuning',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash 001 Tuning',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=16384,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [6]:
import random

name = f'generate-num-{random.randint(0,10000)}'
operation = genai.create_tuned_model(
    # You can use a tuned model here too. Set `source_model="tunedModels/..."`
    source_model=base_model.name,
    training_data=[
        {
             'text_input': '1',
             'output': '2',
        },{
             'text_input': '3',
             'output': '4',
        },{
             'text_input': '-3',
             'output': '-2',
        },{
             'text_input': 'twenty two',
             'output': 'twenty three',
        },{
             'text_input': 'two hundred',
             'output': 'two hundred one',
        },{
             'text_input': 'ninety nine',
             'output': 'one hundred',
        },{
             'text_input': '8',
             'output': '9',
        },{
             'text_input': '-98',
             'output': '-97',
        },{
             'text_input': '1,000',
             'output': '1,001',
        },{
             'text_input': '10,100,000',
             'output': '10,100,001',
        },{
             'text_input': 'thirteen',
             'output': 'fourteen',
        },{
             'text_input': 'eighty',
             'output': 'eighty one',
        },{
             'text_input': 'one',
             'output': 'two',
        },{
             'text_input': 'three',
             'output': 'four',
        },{
             'text_input': 'seven',
             'output': 'eight',
        }
    ],
    id = name,
    epoch_count = 100,
    batch_size=4,
    learning_rate=0.001,
)

## <b><font color='darkblue'>Appendix</font></b>

### <b><font color='darkgreen'>Making authenticated REST calls from Colab</font></b>
In general, you should use the Python SDK to interact with the Gemini API when possible. This example shows how to make OAuth authenticated REST calls from Python for debugging or testing purposes. It assumes you have already set application default credentials from the Quickstart.

In [15]:
import requests

#access_token = !gcloud auth application-default print-access-token
access_token = '...'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token}',
}

headers

{'Content-Type': 'application/json', 'Authorization': 'Bearer ...'}

In [14]:
response = requests.get('https://generativelanguage.googleapis.com/v1/models', headers=headers)
response_json = response.json()
print(response_json)

{'error': {'code': 403, 'message': 'Request had insufficient authentication scopes.', 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.DebugInfo', 'detail': 'Authentication error: 2; Error Details: {AuthType:7,ErrorCode:2,ServiceID:0,Scopes:[],BackendType:2,DebugInfo:gaia_mint_exchange::SCOPE_NOT_PERMITTED: None of the authentication credential token scopes [API_CLOUD_PLATFORM,API_DRIVE,API_EMAIL,API_EMERALDSEA_ME] provided any of the desired scopes [API_GAIA_FILTER_BASE,API_GENERATIVE_LANGUAGE,API_GENERATIVE_LANGUAGE_TUNING,API_GENERATIVE_LANGUAGE_TUNING_READONLY,API_GENERATIVE_LANGUAGE_RETRIEVER,API_GENERATIVE_LANGUAGE_RETRIEVER_READONLY]}'}, {'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_SCOPE_INSUFFICIENT', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com', 'method': 'google.ai.generativelanguage.v1.ModelService.ListModels'}}]}}


In [ ]:
# All the model names
for model in response_json['models']:
    print(model['name'])